In [181]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-f7l8k2s9
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-f7l8k2s9
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Successfully uninstalled urllib3-1.26.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.5.0 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.


In [182]:
!pip install git+https://github.com/Joeclinton1/google-images-download.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Joeclinton1/google-images-download.git to /tmp/pip-req-build-jhwgikz6
  Running command git clone -q https://github.com/Joeclinton1/google-images-download.git /tmp/pip-req-build-jhwgikz6
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
botocore 1.18.18 requires urllib3<1.26,>=1.20; python_version != "3.4", but you have urllib3 1.26.12 which is incompatible.


In [183]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [184]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [185]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [186]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [187]:
## GPU
device = torch.device("cuda:0")

In [188]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [189]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [190]:
import pandas as pd
dataset_train = pd.read_excel('/content/drive/MyDrive/감성대화/감성대화말뭉치(최종데이터)_Training.xlsx', index_col='번호')
dataset_test = pd.read_excel('/content/drive/MyDrive/감성대화/감성대화말뭉치(최종데이터)_Validation.xlsx', index_col='번호')

In [191]:
len(dataset_train), len(dataset_test)

(40879, 5130)

In [192]:
dataset_train.sample(10)

,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
번호,,,,,,,,,,,,,,
26402,청년,여성,대인관계,해당없음,상처,상처,나는 느긋한 편인데 주변 사람들은 답답해 해.,나는 느긋한 편인데 주변 사람들은 답답해 하는군요. 좀 더 자세히 말씀해 주시겠어요?,밥 먹는 것도 느리고 모든 행동이 느리대.,각자의 속도가 있는데 속상하시겠어요.,맞아. 난 지금 이대로가 좋은데 말이야.,사람들의 말이 상처가 되시겠어요.,NaN,NaN
31598,중년,남성,건강,해당없음,당황,열등감,얼마 전에 헬스장에서 운동을 하다 다쳤어. 옆의 젊은 친구가 신경 쓰여서 페이스를 ...,헬스장에서 무리하다가 다치셨군요. 속상하시겠어요.,다친 건 괜찮은데 괜히 자격지심이 들더라고. 몸이 생각처럼 안 따라주니 기운 빠져.,힘드시겠어요. 어떻게 하면 기운이 날까요?,쉬면서 자식들이랑 시간을 좀 보내면 힘이 날 것 같네.,자제 분들과 좋은 재충전 시간이 되었으면 좋겠네요.,NaN,NaN
34031,청소년,남성,학교폭력/따돌림,해당없음,분노,툴툴대는,선생님이 친구들과 사이 좋게 지내세요 라고 얘기하는 걸 들으니 화가나.,친구랑 사이 좋게 지내라는 말에 왜 화가 났는지 조금 더 자세하게 이야기해주세요.,이미 따돌림을 받고 있는 애한테 그런 얘길 하는 게 무슨 소용이람.,이미 따돌림 당하는 친구에게 그런 말을 해서 화가 나셨군요. 이 상황에서 할 수 있...,아무래도 학급회의시간에 학교폭력 예방에 대한 논의를 해봐야겠어.,학급회의시간에 잘 논의되었으면 좋겠어요.,NaN,NaN
28817,노년,여성,대인관계,만성질환 유,기쁨,안도,다시 입원했는데 저번에 잘 돌봐준 담당 간호사라 좋았어.,잘 돌봐준 간호사가 배정되어 좋으시군요.,예의도 바르고 싹싹한 사람이야.,예의도 바르고 싹싹한 사람이군요. 좋은 인연이 있어 행복하시겠어요.,입원해 있는 동안 좋은 말동무가 될 것 같아.,말동무와 행복한 추억을 만들었으면 좋겠어요.,NaN,NaN
12212,중년,남성,"대인관계(부부, 자녀)",해당없음,당황,혼란스러운,요즘 너무 아내의 스킨십이 없어서 멀어진 느낌이야. 나한테 뭐 문제 있나 걱정되네.,혹시 부인께서 언제부터 그런 식으로 반응하셨나요?,내가 출장 다녀온 날부터 그러네. 뭔가 심경의 변화가 생긴 것 같은데 짚이는 게 없...,아내의 달라진 태도의 원인이 뭔지 잘 모르시는 상황이군요?,NaN,NaN,NaN,NaN
9781,청소년,남성,학업 및 진로,해당없음,기쁨,편안한,내가 원하는 성적보다 높게 나와서 마음이 편안해.,생각보다 성적이 높게 나와 마음이 편안하시군요.,걱정했는데 너무 잘 됐다!,지금 이 기분을 유지시킬 방법이 있나요?,주변에 자랑을 하면 이 기분이 유지될 것 같아.,주변에 자랑하면 기분이 유지되시는군요.,NaN,NaN
46962,노년,여성,재정,만성질환 유,분노,구역질 나는,돈이 바닥나서 구역질이 날 정도야. 내 나이 예순살인데 사는게 팍팍해.,예순인 나이에 돈이 없어서 정말 생활이 힘드시겠어요.,기본적인 생활이 어려워서 힘들어.,돈이 없어 기본적인 생활이 힘드시군요.,NaN,NaN,NaN,NaN
6587,중년,여성,"대인관계(부부, 자녀)",해당없음,상처,배신당한,아들이 지금까지 성적을 속여왔어. 아들한테 배신당한 것 같아.,솔직하지 못한 아들로 인해서 배신감이 느껴지시는군요. 이럴 때 아들에게 어떤 생각이...,솔직히 처음엔 많이 황당하고 화가 났는데 자기 나름의 사정이 있었을 거라고 생각돼.,아들 나름대로 속 사정이 있다는 걸 이해하시는군요. 혹시 아들에게 어떤 말을 해보면...,아들의 학업 스트레스를 그동안 너무 몰라준 것 같아서 속 시원하게 털어놓고 얘기하는...,아들과의 대화로 인해 문제를 해결하려고 하시는군요,NaN,NaN
14787,중년,여성,"재정, 은퇴, 노후준비",해당없음,불안,불안,집 살 때 낸 대출을 갚는 데 돈을 다 써서 노후가 불안해.,대출금이 사용자님의 노후에 부담이 되었군요.,그렇지. 금리가 예전보다 낮아져서 부담은 줄었지만 그래도 원금이 많이 남아있어.,이런 문제를 해결할 방법이 있을까요?,대출 상담사나 은행을 찾아가서 상담을 받아야겠어.,전문적인 상담을 받는 방법이 있군요.,NaN,NaN


In [193]:
dataset_train['감정_대분류'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [194]:
dataset_train['감정_대분류'] = dataset_train['감정_대분류'].astype(str).str.replace(' ', '')
dataset_test['감정_대분류'] = dataset_test['감정_대분류'].astype(str).str.replace(' ', '')

In [195]:
dataset_train['감정_대분류'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처'], dtype=object)

In [196]:
dataset_test['감정_대분류'].unique()

array(['분노', '슬픔', '불안', '당황', '상처', '기쁨'], dtype=object)

In [197]:
dataset_train = dataset_train[['감정_대분류','사람문장1','사람문장2','사람문장3']]
dataset_train.head()

,감정_대분류,사람문장1,사람문장2,사람문장3
번호,,,,
44164,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.,아 지금 정말 신이 나.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
3926,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.
50882,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.
31303,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,회사생활을 열심히 해서 계속 월급을 받을거야!
47200,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,NaN


In [198]:
dataset_train.columns = ['Emotion','Sentence1','Sentence2','Sentence3']
dataset_train.head()

,Emotion,Sentence1,Sentence2,Sentence3
번호,,,,
44164,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.,아 지금 정말 신이 나.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
3926,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.
50882,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.
31303,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,회사생활을 열심히 해서 계속 월급을 받을거야!
47200,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,NaN


In [199]:
# 기쁨 0, 불안 1, 당황 2, 슬픔 3, 분노 4, 상처 5
dataset_train.loc[(dataset_train['Emotion'] == "기쁨"), 'Emotion'] = 0
dataset_train.loc[(dataset_train['Emotion'] == "불안"), 'Emotion'] = 1
dataset_train.loc[(dataset_train['Emotion'] == "당황"), 'Emotion'] = 2
dataset_train.loc[(dataset_train['Emotion'] == "슬픔"), 'Emotion'] = 3
dataset_train.loc[(dataset_train['Emotion'] == "분노"), 'Emotion'] = 4
dataset_train.loc[(dataset_train['Emotion'] == "상처"), 'Emotion'] = 5

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [200]:
data_list = []
for a, label in zip(dataset_train['Sentence1'], dataset_train['Emotion']):
  data = []
  data.append(a)
  data.append(str(label))

  data_list.append(data)

In [201]:
print(data_list[6000])

['주변 사람들이 나에게 무능하다고 말해서 씁쓸해.', '3']


In [202]:
dataset_test = dataset_test[['감정_대분류','사람문장1','사람문장2','사람문장3']]
dataset_test.head()

,감정_대분류,사람문장1,사람문장2,사람문장3
번호,,,,
23093,분노,요즘 부모님과 많이 부딪혀.,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.
32848,슬픔,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...,일단 좀 자야겠어.
35590,불안,학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어.,그런데 말을 하고 난 후 나도 괴롭힐까 봐 불안했어.,나는 옳은 일을 한 거라고 생각해. 나까지 괴롭힌다면 신고할 거야.
169,당황,이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야.,곧 있으면 인턴에서 정규직 전환 평가가 있는데 안 좋을 것 같아.,우선 잘못된 부분을 솔직하게 말씀드리고 수정하기 위해서 최선을 다해봐야겠어.
38435,분노,남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어.,어떻게 그럴 수가 있지? 너무 화가 나.,나도 변호사를 고용해서 한번 얘기해보려고 해.


In [203]:
dataset_test.columns = ['Emotion','Sentence1','Sentence2','Sentence3']
dataset_test.head()

,Emotion,Sentence1,Sentence2,Sentence3
번호,,,,
23093,분노,요즘 부모님과 많이 부딪혀.,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.
32848,슬픔,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...,일단 좀 자야겠어.
35590,불안,학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어.,그런데 말을 하고 난 후 나도 괴롭힐까 봐 불안했어.,나는 옳은 일을 한 거라고 생각해. 나까지 괴롭힌다면 신고할 거야.
169,당황,이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야.,곧 있으면 인턴에서 정규직 전환 평가가 있는데 안 좋을 것 같아.,우선 잘못된 부분을 솔직하게 말씀드리고 수정하기 위해서 최선을 다해봐야겠어.
38435,분노,남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어.,어떻게 그럴 수가 있지? 너무 화가 나.,나도 변호사를 고용해서 한번 얘기해보려고 해.


In [204]:
# 기쁨 0, 불안 1, 당황 2, 슬픔 3, 분노 4, 상처 5
dataset_test.loc[(dataset_test['Emotion'] == "기쁨"), 'Emotion'] = 0
dataset_test.loc[(dataset_test['Emotion'] == "불안"), 'Emotion'] = 1
dataset_test.loc[(dataset_test['Emotion'] == "당황"), 'Emotion'] = 2
dataset_test.loc[(dataset_test['Emotion'] == "슬픔"), 'Emotion'] = 3
dataset_test.loc[(dataset_test['Emotion'] == "분노"), 'Emotion'] = 4
dataset_test.loc[(dataset_test['Emotion'] == "상처"), 'Emotion'] = 5

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [205]:
data_list_test = []
for aa, label2 in zip(dataset_test['Sentence1'], dataset_test['Emotion']):
  data2 = []
  data2.append(aa)
  data2.append(str(label2))

  data_list_test.append(data2)

In [206]:
print(data_list_test[20])

['학교폭력 가해자로 낙인찍혀서 우리 부모님까지 손가락질을 당했어.', '2']


In [207]:
dataset_train = data_list
dataset_test = data_list_test

In [208]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [209]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [210]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower= False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [211]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [212]:
data_train[0]

(array([   2, 3096, 5330, 1788, 5947, 6855, 4575, 7784, 1767, 6553, 4102,
        3010, 7096, 1370,  517,   54,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(17, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [213]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [214]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [215]:
from transformers import BertForSequenceClassification

In [216]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [217]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [218]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [219]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [220]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [221]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [225]:
train_dataloader

In [226]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/639 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8667958974838257 train acc 0.140625
epoch 1 batch id 201 loss 1.3022150993347168 train acc 0.2764303482587065
epoch 1 batch id 401 loss 1.1175236701965332 train acc 0.42019950124688277
epoch 1 batch id 601 loss 0.8308350443840027 train acc 0.48206114808652245
epoch 1 train acc 0.4904932699031066


  0%|          | 0/81 [00:00<?, ?it/s]

epoch 1 test acc 0.6222993827160493


  0%|          | 0/639 [00:00<?, ?it/s]

...

epoch 9 test acc 0.645216049382716


  0%|          | 0/639 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.14781513810157776 train acc 0.953125
epoch 10 batch id 201 loss 0.13655194640159607 train acc 0.9314365671641791
epoch 10 batch id 401 loss 0.2630602717399597 train acc 0.9344217581047382
epoch 10 batch id 601 loss 0.09733760356903076 train acc 0.9360440931780366
epoch 10 train acc 0.9361707330270036


  0%|          | 0/81 [00:00<?, ?it/s]

epoch 10 test acc 0.6450231481481482


In [227]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower= False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [228]:
def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length = valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]
    for i in out:
      logits=i
      logits = logits.detach().cpu().numpy()

      if np.argmax(logits) == 0:
        test_eval.append("기쁨")
      elif np.argmax(logits) == 1:
        test_eval.append("불안")
      elif np.argmax(logits) == 2:
        test_eval.append("당황")
      elif np.argmax(logits) == 3:
        test_eval.append("슬픔")
      elif np.argmax(logits) == 4:
        test_eval.append("분노")
      elif np.argmax(logits) == 5:
        test_eval.append("상처")

    print("오늘 느끼는 감성 키워드는 : "+test_eval[0] +" 입니다.")

In [ ]:
end = 1
while end == 1 :
  sentence = input("오늘 어땠나요? ")

  if sentence == "없음" :
    break
  predict(sentence)

  print('\n')

In [230]:
from google_images_download import google_images_download 

In [246]:
def googleImageCrawling(keyword, limit):
    response = google_images_download.googleimagesdownload()  

    arguments = {"keywords": keyword, "limit":limit, "print_urls":True, "chromedriver": "./Users/eunkyoungseo/code/test/.wdm/drivers/chromedriver/mac64/106.0.5249/chromedriver", "format":"jpg"}  
    paths = response.download(arguments)  
    print(paths)

print("<예시>\n기쁠 때 생각 나는 음식\n불안할 때 먹으면 좋은 음식\n당황할 때 먹으면 좋은 음식\n슬플 때 먹으면 좋은 음식\n분노할 때 먹으면 좋은 음식\n상처 받았을 때 먹으면 좋은 음식\n")
keyword = input("검색 키워드를 입력해주세요 : ")

limit = 4
googleImageCrawling(keyword, int(limit))

<예시>
기쁠 때 생각 나는 음식
불안할 때 먹으면 좋은 음식
당황할 때 먹으면 좋은 음식
슬플 때 먹으면 좋은 음식
분노할 때 먹으면 좋은 음식
상처 받았을 때 먹으면 좋은 음식

검색 키워드를 입력해주세요 : 슬플 때 먹으면 좋은 음식

Item no.: 1 --> Item name = \uc2ac\ud50c \ub54c \uba39\uc73c\uba74 \uc88b\uc740 \uc74c\uc2dd
Evaluating...
Starting Download...
Image URL: https://images.chosun.com/resizer/G2j6SkWuUt4Zs70v5P3putWDOtU=/640x345/smart/cloudfront-ap-northeast-1.images.arcpublishing.com/chosun/WVH2RKECTEZ52UPYF5WGI4ITRI.jpg
Completed Image ====> 1.wvh2rkectez52upyf5wgi4itri.jpg
Image URL: https://s3.ap-northeast-2.amazonaws.com/img.kormedi.com/news/article/__icsFiles/artimage/2015/08/24/c_km601/food_540.jpg
Invalid image format 'binary/octet-stream'. Skipping...
Image URL: https://cloudfront-ap-northeast-1.images.arcpublishing.com/chosun/JQQPJSOTWIB7ADEMB4NJBN4WK4.jpg
Completed Image ====> 2.jqqpjsotwib7ademb4njbn4wk4.jpg
Image URL: https://mblogthumb-phinf.pstatic.net/MjAxNzA4MjJfMjM1/MDAxNTAzMzg0OTM3MjIy.BEliAr3DTbWCWdI_88UF3aq-p-h8gLmA4jJF8MpnVDEg.VOaPx4BxarM0rnOWfq4PKJ-